In [1]:
import pandas as pd
import os

In [21]:
files = [
    "source/csv/LEPTBR24.csv",
    "source/csv/LEPTBR23.csv",
    "source/csv/LEPTBR22.csv",
    "source/csv/LEPTBR21.csv",
    "source/csv/LEPTBR20.csv",
]

dfs = []
for file in files:
    dfs.append(pd.read_csv(file, usecols=['DT_NOTIFIC', 'EVOLUCAO', 'ATE_DT_ALT']))

linhasAntes = []
for df in dfs:
    df["ATE_DT_ALT"] = pd.to_datetime(df["ATE_DT_ALT"])
    df["DT_NOTIFIC"] = pd.to_datetime(df["DT_NOTIFIC"])
    linhasAntes.append(df.shape[0])

print(dfs)

[      DT_NOTIFIC ATE_DT_ALT  EVOLUCAO
0     2024-01-01 2024-01-06       1.0
1     2024-01-01        NaT       NaN
2     2024-01-01        NaT       NaN
3     2024-01-01        NaT       1.0
4     2024-01-01        NaT       9.0
...          ...        ...       ...
14237 2024-06-19        NaT       NaN
14238 2024-06-19        NaT       NaN
14239 2024-06-19        NaT       NaN
14240 2024-06-19        NaT       NaN
14241 2024-06-20        NaT       NaN

[14242 rows x 3 columns],       DT_NOTIFIC ATE_DT_ALT  EVOLUCAO
0     2023-01-01        NaT       NaN
1     2023-01-01 2023-01-01       NaN
2     2023-01-01        NaT       9.0
3     2023-01-01 2023-01-02       1.0
4     2023-01-01        NaT       1.0
...          ...        ...       ...
20229 2023-12-31 2023-12-31       1.0
20230 2023-12-31        NaT       1.0
20231 2023-12-31        NaT       1.0
20232 2023-12-31        NaT       9.0
20233 2023-12-31        NaT       1.0

[20234 rows x 3 columns],       DT_NOTIFIC ATE_DT_ALT  EVOL

In [33]:
linhasDepois = []
for i, df in enumerate(dfs):
    # Modificar o df
    df = df.dropna(subset=["EVOLUCAO", "ATE_DT_ALT", "DT_NOTIFIC"])
    df = df[df["EVOLUCAO"] != 9.0]
    
    # Atualizar a lista dfs com o DataFrame alterado
    dfs[i] = df  # Substitui o DataFrame modificado na lista
    
    linhasDepois.append(df.shape[0])

for i in range(len(files)):
    print(f"{(1 - (linhasDepois[i] / linhasAntes[i])) * 100:.2f}% das linhas foram removidas por falta de dados no arquivo {os.path.basename(files[i])}")



86.88% das linhas foram removidas por falta de dados no arquivo LEPTBR24.csv
79.72% das linhas foram removidas por falta de dados no arquivo LEPTBR23.csv
77.97% das linhas foram removidas por falta de dados no arquivo LEPTBR22.csv
76.54% das linhas foram removidas por falta de dados no arquivo LEPTBR21.csv
77.46% das linhas foram removidas por falta de dados no arquivo LEPTBR20.csv


In [34]:
medias = []
medianas = []

for i, df in enumerate(dfs):
    df["duracao"] = (df["ATE_DT_ALT"] - df["DT_NOTIFIC"]).dt.days
    media = df["duracao"].mean()
    mediana = df['duracao'].median()
    medias.append(media)
    medianas.append(mediana)
    dfs[i] = df
    print(f"A média de tempo entre alta e notificação é: {media:.1f}")


A média de tempo entre alta e notificação é: 5.4
A média de tempo entre alta e notificação é: 5.9
A média de tempo entre alta e notificação é: 6.5
A média de tempo entre alta e notificação é: 5.4
A média de tempo entre alta e notificação é: 6.3


In [40]:
mortes_lept = []
mortes_outro = []
totais = []

for df in dfs:
    counts = df.groupby("EVOLUCAO").size()
    total = counts.sum()
    print(counts)
    morte_lept = counts[2.0]
    morte_outro = counts[3.0]
    totais.append(total)
    mortes_lept.append(morte_lept)
    mortes_outro.append(morte_outro)
    print(f"Total de registros: {total}")
    print(f"Mortes por leptospirose: {morte_lept}")
    print(f"Mortes por outras causas: {morte_outro}")
    print(f"Soma das mortes: {morte_lept + morte_outro}")
    print(f"Taxa de mortalidade total: {(morte_lept + morte_outro) / total * 100:.2f}%")
    print("---")

EVOLUCAO
1.0    1606
2.0      73
3.0     190
dtype: int64
Total de registros: 1869
Mortes por leptospirose: 73
Mortes por outras causas: 190
Soma das mortes: 263
Taxa de mortalidade total: 14.07%
---
EVOLUCAO
1.0    3601
2.0     102
3.0     400
dtype: int64
Total de registros: 4103
Mortes por leptospirose: 102
Mortes por outras causas: 400
Soma das mortes: 502
Taxa de mortalidade total: 12.23%
---
EVOLUCAO
1.0    2819
2.0     103
3.0     332
dtype: int64
Total de registros: 3254
Mortes por leptospirose: 103
Mortes por outras causas: 332
Soma das mortes: 435
Taxa de mortalidade total: 13.37%
---
EVOLUCAO
1.0    1731
2.0      69
3.0     232
dtype: int64
Total de registros: 2032
Mortes por leptospirose: 69
Mortes por outras causas: 232
Soma das mortes: 301
Taxa de mortalidade total: 14.81%
---
EVOLUCAO
1.0    1931
2.0      51
3.0     223
dtype: int64
Total de registros: 2205
Mortes por leptospirose: 51
Mortes por outras causas: 223
Soma das mortes: 274
Taxa de mortalidade total: 12.43%
--

In [44]:
for i in range(len(files)):
    print(f"Sobre {os.path.basename(files[i])}: ")
    print(f"{(1-(linhasDepois[i]/linhasAntes[i]))*100:.2f}% das linhas foram removidas por falta de dados.")
    print(f"A média de tempo entre alta e notificação é: {medias[i]:.1f}")
    print(f"A mediana de tempo entre alta e notificação é: {medianas[i]:.1f}")
    print(f"Total de registros após limpeza: {totais[i]}")
    print(f"Mortes por leptospirose: {mortes_lept[i]}")
    print(f"Mortes por outras causas: {mortes_outro[i]}")
    print(f"Soma das mortes: {mortes_lept[i] + mortes_outro[i]}")
    print(f"Taxa de mortalidade total: {(mortes_lept[i] + mortes_outro[i]) / totais[i] * 100:.2f}%")
    
    print("-------------------------------")
    print("-------------------------------")
    print("-------------------------------")

Sobre LEPTBR24.csv: 
86.88% das linhas foram removidas por falta de dados.
A média de tempo entre alta e notificação é: 5.4
A mediana de tempo entre alta e notificação é: 4.0
Total de registros após limpeza: 1869
Mortes por leptospirose: 73
Mortes por outras causas: 190
Soma das mortes: 263
Taxa de mortalidade total: 14.07%
-------------------------------
-------------------------------
-------------------------------
Sobre LEPTBR23.csv: 
79.72% das linhas foram removidas por falta de dados.
A média de tempo entre alta e notificação é: 5.9
A mediana de tempo entre alta e notificação é: 4.0
Total de registros após limpeza: 4103
Mortes por leptospirose: 102
Mortes por outras causas: 400
Soma das mortes: 502
Taxa de mortalidade total: 12.23%
-------------------------------
-------------------------------
-------------------------------
Sobre LEPTBR22.csv: 
77.97% das linhas foram removidas por falta de dados.
A média de tempo entre alta e notificação é: 6.5
A mediana de tempo entre alta e